In [ ]:
import pandas as pd
import pandas.io.sql as psql
import os

from sqlalchemy import create_engine

In [ ]:
# URL format: 'postgres+psycopg2://{user}:{password}@{host:port}/{dbname}'
LOGISTICS_DWH_URL = os.environ["LOGISTICS_DWH_URL"]
PANDORA_DWH_URL = os.environ["PANDORA_DWH_URL"]
PANDORA_DWH_EXTERNAL_URL = os.environ["PANDORA_DWH_EXTERNAL_URL"]

In [ ]:
dwh_engine = create_engine(LOGISTICS_DWH_URL, echo=False, use_batch_mode=True)

dwh_query = """
select o.id, o.created_at, o.updated_at
from legacy_merge_layer.hurrier_orders o
where o.country_code = 'th'
"""

dwh_data = pd.read_sql_query(dwh_query, dwh_engine, index_col='id', chunksize=200000)

In [ ]:
pandora_engine = create_engine(PANDORA_DWH_URL, echo=False, use_batch_mode=True)

pandora_query = """
select o.id, o.created_at, o.updated_at
from merge_layer_rdbms_hurrier.orders o
inner join dm_ops_fo.dim_countries c using (rdbms_id)
where c.country_iso = 'TH'
"""

pandora_data = pd.read_sql_query(pandora_query, pandora_engine, index_col='id', chunksize=200000)

In [ ]:
# pd.merge(dwh_data, pandora_data, on='id', how='right', suffixes=('_logistics', '_pandora'))

In [ ]:
# Identify what values are in pandora_data and not in dwh_data
key_diff = set(pandora_data.Key).difference(dwh_data.Key)
pandora_data.Key.isin(key_diff)